In [1]:
import json
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import HTML, display
import tabulate
import sqlite3
from sqlite3 import Error
import random
import copy
import math


human_labels = ('woman', 'men', 'boy', 'kid', 'child', 'guy', 'man', 'person', 'girl', 'lady', 'people')

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('Eurocity',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_eurocity_df = pd.DataFrame(table, columns =head) 
r_eurocity_df

,Object_1,Relation,Object_2,Count
0,building,behind,man,13315
1,building,behind,person,11261
2,person,walking on,street,8611
3,person,walking on,sidewalk,7407
4,man,walking on,street,6689
...,...,...,...,...
677,woman,watching,boy,1
678,woman,watching,dog,1
679,woman,watching,light,1
680,woman,watching,umbrella,1


In [4]:
r_eurocity_df

,Object_1,Relation,Object_2,Count
0,building,behind,man,13315
1,building,behind,person,11261
2,person,walking on,street,8611
3,person,walking on,sidewalk,7407
4,man,walking on,street,6689
...,...,...,...,...
677,woman,watching,boy,1
678,woman,watching,dog,1
679,woman,watching,light,1
680,woman,watching,umbrella,1


In [6]:
r_eurocity_df['images'] = ""
# import json
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()

# rel_dict = dict()

for index, row in r_eurocity_df.iterrows():
# for index, row in tqdm_notebook(r_eurocity_df.iterrows()):
#     print(f"{r_eurocity_df.at[index, 'Object_1']}, {r_eurocity_df.at[index, 'Relation']}, {r_eurocity_df.at[index, 'Object_2']}")
    q_in = ('Eurocity' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
#         im_set.add(r)
    r_eurocity_df.at[index, 'images'] = list(t_st)

r_eurocity_df

,Object_1,Relation,Object_2,Count,images
0,building,behind,man,13315,"[pisa_00073, prague_00988, dresden_00372, mars..."
1,building,behind,person,11261,"[marseille_00950, toulouse_00264, budapest_006..."
2,person,walking on,street,8611,"[marseille_00950, toulouse_00264, budapest_006..."
3,person,walking on,sidewalk,7407,"[bologna_00250, roma_00936, marseille_00950, b..."
4,man,walking on,street,6689,"[prague_00264, pisa_00073, amsterdam_01004, mi..."
...,...,...,...,...,...
677,woman,watching,boy,1,[firenze_00429]
678,woman,watching,dog,1,[milano_00406]
679,woman,watching,light,1,[budapest_00267]
680,woman,watching,umbrella,1,[nuernberg_00039]


In [7]:
im_list = r_eurocity_df.images.to_list()

In [8]:
len(set(im_list[677]))

1

In [9]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('Caltech',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_caltech_df = pd.DataFrame(table, columns =head) 

In [10]:
r_caltech_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()

rel_dict = dict()
for index, row in r_caltech_df.iterrows():
    q_in = ('Caltech' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_caltech_df.at[index, 'images'] = list(t_st)

r_caltech_df

,Object_1,Relation,Object_2,Count,images
0,person,walking on,street,2734,"[set04_V002_1717, set01_V004_1374, set00_V014_..."
1,person,walking on,sidewalk,2277,"[set04_V002_1717, set03_V008_599, set01_V004_1..."
2,building,behind,person,1629,"[set04_V002_1717, set00_V002_361, set01_V004_2..."
3,man,walking on,street,1456,"[set04_V002_1717, set01_V004_1374, set00_V014_..."
4,tree,behind,person,1329,"[set04_V002_1717, set04_V008_1119, set01_V004_..."
...,...,...,...,...,...
551,woman,walking on,fence,1,[set04_V005_1638]
552,woman,watching,boy,1,[set00_V013_19]
553,woman,watching,car,1,[set00_V008_1476]
554,woman,watching,light,1,[set02_V010_494]


In [11]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('CityPerson',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_cityperson_df = pd.DataFrame(table, columns =head) 
r_cityperson_df 

,Object_1,Relation,Object_2,Count
0,building,behind,man,1746
1,building,behind,person,1239
2,person,walking on,sidewalk,1071
3,person,walking on,street,909
4,tree,behind,man,889
...,...,...,...,...
259,woman,looking at,tree,1
260,woman,riding,motorcycle,1
261,woman,sitting on,bench,1
262,woman,sitting on,chair,1


In [12]:

r_cityperson_df['images'] = ""
database = r"person.db"
im_set = set()
# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in r_cityperson_df.iterrows():
    q_in = ('CityPerson' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
        im_set.add(r[0])
#         for el in r:
#             im_set.add(el)
    r_cityperson_df.at[index, 'images'] = list(t_st)

r_cityperson_df
# len(im_set)

,Object_1,Relation,Object_2,Count,images
0,building,behind,man,1746,"[aachen_000140_000019_leftImg8bit, strasbourg_..."
1,building,behind,person,1239,"[aachen_000140_000019_leftImg8bit, monchenglad..."
2,person,walking on,sidewalk,1071,"[zurich_000077_000019_leftImg8bit, cologne_000..."
3,person,walking on,street,909,"[strasbourg_000000_029915_leftImg8bit, zurich_..."
4,tree,behind,man,889,"[aachen_000140_000019_leftImg8bit, weimar_0001..."
...,...,...,...,...,...
259,woman,looking at,tree,1,[bochum_000000_009554_leftImg8bit]
260,woman,riding,motorcycle,1,[dusseldorf_000165_000019_leftImg8bit]
261,woman,sitting on,bench,1,[hamburg_000000_098061_leftImg8bit]
262,woman,sitting on,chair,1,[hamburg_000000_052122_leftImg8bit]


In [13]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('CrowdHuman',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_crowdhuman_df = pd.DataFrame(table, columns =head) 
# crowdhuman_df 

In [14]:
r_crowdhuman_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in r_crowdhuman_df.iterrows():
    q_in = ('CrowdHuman' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_crowdhuman_df.at[index, 'images'] = list(t_st)

r_crowdhuman_df

,Object_1,Relation,Object_2,Count,images
0,man,watching,man,14390,"[283554,2d29e0002a1f82bc, 273278,18a02000baf60..."
1,person,watching,man,12557,"[282555,e931a000e28f6736, 273278,18a02000baf60..."
2,building,behind,man,10467,"[283991,1e6db0005787f070, 273275,13523000e293f..."
3,man,wearing,shirt,8989,"[273278,97d7f00040d24761, 273278,18a02000baf60..."
4,building,behind,woman,8893,"[273278,d55e80001820386b, 283554,190fa0000ed91..."
...,...,...,...,...,...
1215,woman,watching,kid,1,"[284193,1b30f00097a44d6f]"
1216,woman,watching,letter,1,"[273275,13d0a00096a29505]"
1217,woman,watching,skier,1,"[282555,2324f000246e605b]"
1218,woman,watching,tail,1,"[273278,34dfc00090520457]"


In [15]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('WiderPerson',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_widerperson_df = pd.DataFrame(table, columns=head) 
r_widerperson_df 

,Object_1,Relation,Object_2,Count
0,person,watching,man,11914
1,man,watching,man,9635
2,person,watching,woman,7488
3,building,behind,man,5864
4,person,wearing,shirt,4921
...,...,...,...,...
593,woman,holding,bottle,1
594,woman,holding,kite,1
595,woman,under,umbrella,1
596,woman,watching,cow,1


In [16]:
r_widerperson_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in r_widerperson_df.iterrows():
    q_in = ('WiderPerson' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_widerperson_df.at[index, 'images'] = list(t_st)

r_widerperson_df

,Object_1,Relation,Object_2,Count,images
0,person,watching,man,11914,"[014779, 008002, 015392, 010801, 009860, 01796..."
1,man,watching,man,9635,"[000417, 010151, 010801, 009860, 017967, 02053..."
2,person,watching,woman,7488,"[012916, 013018, 006539, 010801, 000372, 00524..."
3,building,behind,man,5864,"[014779, 012916, 013018, 010151, 008002, 00037..."
4,person,wearing,shirt,4921,"[001886, 007904, 003440, 006032, 014160, 00440..."
...,...,...,...,...,...
593,woman,holding,bottle,1,[009185]
594,woman,holding,kite,1,[018589]
595,woman,under,umbrella,1,[010318]
596,woman,watching,cow,1,[013810]


In [18]:
def setA_setB(A, B):
    """
    set A - set B
    """
    common_list = []
    diff_set = []
    comm_set = []
    for idx, row in A.iterrows():
        index_list = B[(B['Object_1'] == row['Object_1'])&(B['Relation'] == row['Relation'])&(B['Object_2'] == row['Object_2'])].index.tolist()
        if not index_list:
           diff_set.append(idx) 
        else:
           comm_set.append(idx)
    diff_cp = A.iloc[diff_set]
    com_cp = A.iloc[comm_set]
    return diff_cp, com_cp

def find_quantile(diff_set,q=0.95, max_im=20):
    quantile_95 = diff_set[diff_set.Count > diff_set.Count.quantile(q)]
    quantile_95 = quantile_95.sort_values('Count', ascending=False)
    quantile_95.reset_index(drop=True, inplace=True)
    quantile_95['count_down'] = max_im
#     im_to_add = []
    for index, row in quantile_95.iterrows():
        if len(quantile_95.at[index, 'images']) > quantile_95.at[index, 'count_down']:
            s_images = random.sample(quantile_95.at[index, 'images'], quantile_95.at[index, 'count_down'])
            quantile_95.at[index, 'images'] = s_images
        else:
            s_images = quantile_95.at[index, 'images']
        quantile_95.at[index, 'count_down'] = quantile_95.at[index, 'count_down'] - len(s_images)
#         im_to_add.extend(s_images)

#     im_to_add = list(set(im_to_add))
#     return im_to_add, quantile_95
    return quantile_95

def splite_data(data_df, train_size=0.55, val_size=0.1, test_size=0.35):
    data_df = copy.deepcopy(data_df)
    all_images = list(set([item for sublist in data_df['images'].tolist() for item in sublist]))
    train_images = random.sample(all_images, int(len(all_images) * train_size))
    rem_images = list(set(all_images).difference(set(train_images)))
    val_images = random.sample(rem_images, int(math.ceil(len(all_images) * val_size)))
    test_images = list(set(all_images).difference(set(train_images).union(set(val_images))))
    
#     print(set(train_images).intersection(set(val_images)))

    assert len(all_images) == len(train_images) + len(val_images) + len(test_images)
    assert len(set(train_images).intersection(set(val_images))) == 0
    assert len(set(train_images).intersection(set(test_images))) == 0
    assert len(set(val_images).intersection(set(test_images))) == 0
    return train_images, val_images, test_images

def concat_df(df_list):
    bigdata = copy.deepcopy(df_list[0])
    for el in df_list[1:]:
        bigdata = bigdata.append(el, ignore_index=True)
    return bigdata

def find_index_of_images(img_df, image_name_list=None):
    index_id_list = []
    for el in image_name_list:
        index_ids = img_df.index[img_df['file_name'].str.contains(el)].tolist()
        image_id = [img_df.at[idx, 'id'] for idx in index_ids]
        if len(image_id) != 1:
            print(el, image_id)
        assert len(image_id) == 1
        index_id_list.append(index_ids[0])
    return index_id_list

def create_ann_df(ann_df, index_list):
    selected_ann_df = ann_df[ann_df['image_id'].isin([index_list[0] + 1])]
    for el in index_list[1:]:
        selected_ann_df = pd.concat([selected_ann_df,ann_df[ann_df['image_id'].isin([el + 1])]])
    return selected_ann_df

def create_img_df(img_df, index_list):
    selected_img_df = img_df.loc[[index_list[0]]]
    for el in index_list[1:]:
        selected_img_df = pd.concat([selected_img_df,img_df.loc[[el]]])
    return selected_img_df

def update_img_ids(aug_img_df, row=None, dataset='other'):
    last_id = aug_img_df.at[aug_img_df.index[-1], 'id']
    concat_df = pd.concat([aug_img_df, row])
    concat_df.reset_index(drop=True, inplace=True)
    concat_df.at[concat_df.index[-1], 'id'] = last_id + 1
    return last_id + 1, concat_df

def uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df, dt='other'):
    selected_img_df = selected_img_df.sort_values('id')
    if dt == 'ch':
        selected_ann_df['height'] = ""
        selected_ann_df['width'] = ""
        for idx, row in selected_img_df.iterrows():
            new_image_id, aug_img_df = update_img_ids(aug_img_df, row=selected_img_df.loc[[idx]], dataset='ch')
            
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'height'] = aug_img_df.at[new_image_id -1, 'height']
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'width'] = aug_img_df.at[new_image_id -1, 'width']
            
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'image_id'] = new_image_id

    else:
        for idx, row in selected_img_df.iterrows():
            new_image_id, aug_img_df = update_img_ids(aug_img_df, row=selected_img_df.loc[[idx]])
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'image_id'] = new_image_id
    
    return aug_img_df, selected_ann_df

def concat_ann_df(orig_df, add_df):
    ann_df_c = pd.concat([orig_df, add_df])
    ann_df_c.reset_index(drop=True, inplace=True)
    for idx, row in ann_df_c.iterrows():
        if idx + 1 != row['id']:
            ann_df_c.at[idx, 'id'] = idx + 1
    return ann_df_c

## Find diff between the relation datasets

In [19]:
clt_cp, _ = setA_setB(r_caltech_df, r_cityperson_df)
clt_cp

,Object_1,Relation,Object_2,Count,images
31,people,watching,letter,101,"[set00_V012_1620, set00_V006_1795, set00_V013_..."
32,person,watching,vehicle,100,"[set05_V005_709, set00_V009_1453, set00_V011_4..."
33,person,watching,letter,91,"[set00_V008_846, set00_V007_1275, set00_V006_1..."
37,people,watching,vehicle,83,"[set00_V012_1620, set00_V009_1453, set00_V012_..."
38,person,riding,car,79,"[set02_V010_614, set02_V008_1446, set02_V008_1..."
...,...,...,...,...,...
551,woman,walking on,fence,1,[set04_V005_1638]
552,woman,watching,boy,1,[set00_V013_19]
553,woman,watching,car,1,[set00_V008_1476]
554,woman,watching,light,1,[set02_V010_494]


In [20]:
all_images = list(set([item for sublist in clt_cp['images'].tolist() for item in sublist]))
len(all_images)

565

In [21]:
clt_quantile_95 = find_quantile(clt_cp,q=0.95, max_im=20)
clt_quantile_95

,Object_1,Relation,Object_2,Count,images,count_down
0,people,watching,letter,101,"[set00_V014_458, set00_V009_643, set00_V013_43...",0
1,person,watching,vehicle,100,"[set00_V008_756, set01_V001_1379, set02_V010_3...",0
2,person,watching,letter,91,"[set00_V013_559, set00_V008_756, set00_V007_12...",0
3,people,watching,vehicle,83,"[set00_V011_179, set00_V006_1705, set01_V002_7...",0
4,person,riding,car,79,"[set02_V010_614, set02_V008_1446, set02_V008_1...",3
5,light,above,person,63,"[set00_V012_1680, set00_V008_246, set00_V013_1...",0
6,vehicle,behind,person,59,"[set03_V006_1042, set00_V011_299, set00_V008_1...",0
7,person,watching,truck,49,"[set00_V001_45, set01_V001_1199, set02_V009_14...",0
8,roof,above,person,44,"[set00_V009_1093, set00_V007_345, set02_V009_1...",0
9,light,above,people,33,"[set00_V012_1110, set00_V012_1620, set00_V012_...",0


In [17]:
all_images = list(set([item for sublist in clt_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

121

In [23]:
ecp_cp, _ = setA_setB(r_eurocity_df, r_cityperson_df)
ecp_cp

,Object_1,Relation,Object_2,Count,images
84,people,riding,motorcycle,60,"[roma_00959, pisa_00274, roma_00778, roma_0011..."
85,person,watching,motorcycle,60,"[firenze_00543, roma_00113, dresden_00178, bol..."
87,person,wearing,helmet,55,"[roma_00113, marseille_00789, firenze_00331, f..."
94,sign,above,man,47,"[hamburg_00584, nuernberg_00469, hamburg_00376..."
103,person,in,window,41,"[potsdam_00333, amsterdam_00324, stuttgart_002..."
...,...,...,...,...,...
677,woman,watching,boy,1,[firenze_00429]
678,woman,watching,dog,1,[milano_00406]
679,woman,watching,light,1,[budapest_00267]
680,woman,watching,umbrella,1,[nuernberg_00039]


In [24]:
all_images = list(set([item for sublist in ecp_cp['images'].tolist() for item in sublist]))
len(all_images)

1194

In [25]:
ecp_quantile_95 = find_quantile(ecp_cp,q=0.95, max_im=40)
ecp_quantile_95

,Object_1,Relation,Object_2,Count,images,count_down
0,people,riding,motorcycle,60,"[barcelona_00752, budapest_00435, roma_00220, ...",0
1,person,watching,motorcycle,60,"[amsterdam_00286, marseille_00816, toulouse_00...",0
2,person,wearing,helmet,55,"[milano_00270, roma_00582, firenze_00459, bolo...",0
3,sign,above,man,47,"[hamburg_00584, nuernberg_00469, hamburg_00376...",0
4,person,in,window,41,"[potsdam_00333, amsterdam_00324, stuttgart_002...",22
5,window,above,man,38,"[prague_01342, bologna_00371, montpellier_0022...",9
6,car,near,man,36,"[firenze_00484, firenze_00813, bologna_00098, ...",8
7,window,above,person,35,"[leipzig_00018, zagreb_00149, roma_00891, dres...",13
8,person,wearing,hat,33,"[amsterdam_00424, firenze_00834, budapest_0020...",15
9,person,watching,truck,32,"[budapest_00442, zagreb_00026, milano_00393, b...",10


In [26]:
all_images = list(set([item for sublist in ecp_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

521

In [27]:
ch_cp, _ = setA_setB(r_crowdhuman_df, r_cityperson_df)
ch_cp

,Object_1,Relation,Object_2,Count,images
33,man,wearing,hat,2005,"[282555,43fe0000c2d63da8, 282555,1247c500014eb..."
55,person,watching,girl,1165,"[283647,1df20005baf8961, 273278,58c3c000f2e589..."
62,woman,watching,girl,1015,"[284193,15f68000865294d7, 273278,8fdd70007246e..."
63,man,wearing,sock,978,"[283647,1ab5000066a030c4, 273275,eb64200013294..."
67,person,wearing,hat,911,"[282555,1247c500014eb68f0, 273275,79ccb0009873..."
...,...,...,...,...,...
1215,woman,watching,kid,1,"[284193,1b30f00097a44d6f]"
1216,woman,watching,letter,1,"[273275,13d0a00096a29505]"
1217,woman,watching,skier,1,"[282555,2324f000246e605b]"
1218,woman,watching,tail,1,"[273278,34dfc00090520457]"


In [28]:
all_images = list(set([item for sublist in ch_cp['images'].tolist() for item in sublist]))
len(all_images)

8265

In [29]:
ch_quantile_95 = find_quantile(ch_cp,q=0.95, max_im=20)
ch_quantile_95

,Object_1,Relation,Object_2,Count,images,count_down
0,man,wearing,hat,2005,"[283991,10f4400024b3988e, 273278,b9daf00030bad...",0
1,person,watching,girl,1165,"[273275,6849f0002fb6a1ef, 283554,217c70001c605...",0
2,woman,watching,girl,1015,"[282555,4510f0009bcd43e5, 273278,8404f000e741c...",0
3,man,wearing,sock,978,"[283554,28c10000ce705a1a, 273275,40c3c000afe5f...",0
4,person,wearing,hat,911,"[273275,e2066000a0e134ad, 282555,1247c500014eb...",0
5,woman,watching,boy,878,"[273275,c09240000e89435e, 273278,b1eab00025b0d...",0
6,woman,has,leg,854,"[273275,90636000312920f0, 284193,1f88600039349...",0
7,woman,wearing,boot,815,"[273278,13c1c9000ab30f879, 282555,112aab00010b...",0
8,girl,wearing,shoe,771,"[282555,1042900065b87987, 283554,1278e0005258a...",0
9,man,has,face,761,"[273278,97d650006993bbbb, 282555,dfa0b0005e184...",0


In [30]:
all_images = list(set([item for sublist in ch_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

926

In [31]:
wp_cp, _ = setA_setB(r_widerperson_df, r_cityperson_df)
wp_cp

,Object_1,Relation,Object_2,Count,images
17,person,watching,girl,2478,"[015444, 001014, 007669, 000790, 006539, 01787..."
31,woman,watching,girl,1361,"[015444, 008680, 017949, 000790, 014464, 00052..."
39,woman,watching,boy,1059,"[000790, 017035, 014616, 014977, 000493, 01413..."
50,mountain,behind,person,684,"[006184, 003256, 006081, 005783, 013435, 00914..."
52,mountain,behind,man,641,"[006184, 003256, 006081, 005783, 013435, 00914..."
...,...,...,...,...,...
593,woman,holding,bottle,1,[009185]
594,woman,holding,kite,1,[018589]
595,woman,under,umbrella,1,[010318]
596,woman,watching,cow,1,[013810]


In [32]:
all_images = list(set([item for sublist in wp_cp['images'].tolist() for item in sublist]))
len(all_images)

4303

In [33]:
wp_quantile_95 = find_quantile(wp_cp,q=0.95, max_im=30)
wp_quantile_95

,Object_1,Relation,Object_2,Count,images,count_down
0,person,watching,girl,2478,"[008970, 001043, 016047, 005218, 003140, 02016...",0
1,woman,watching,girl,1361,"[005721, 006660, 004128, 013205, 017464, 00090...",0
2,woman,watching,boy,1059,"[004627, 010579, 012607, 015279, 003026, 00900...",0
3,mountain,behind,person,684,"[004173, 017865, 013363, 009144, 019309, 00521...",0
4,mountain,behind,man,641,"[006569, 019312, 003958, 009514, 007144, 00701...",0
5,people,wearing,short,632,"[004810, 014033, 017390, 003401, 018165, 00887...",0
6,boy,watching,man,517,"[019052, 001738, 019629, 000434, 019480, 00193...",0
7,man,wearing,hat,457,"[005280, 006815, 017139, 005622, 014991, 00308...",0
8,boy,watching,boy,435,"[008097, 003034, 011129, 001738, 016505, 01738...",0
9,boy,wearing,pant,409,"[001037, 007527, 004044, 008245, 006653, 02041...",0


In [34]:
all_images = list(set([item for sublist in wp_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

633

In [35]:
df_list = []
df_list.append(ecp_quantile_95)

In [36]:
ch_q_ecp_q, _ = setA_setB(ch_quantile_95, ecp_quantile_95)
# ch_seclected_df = splite_data(ch_q_ecp_q, train_size=0.55, val_size=0.1, test_size=0.35)
# ch_seclected_df

In [37]:
df_list.append(ch_q_ecp_q)

In [38]:
concat_data = concat_df(df_list)
# concat_data

In [34]:
wp_q_con, _ = setA_setB(wp_quantile_95, concat_data)
# wp_seclected_df = splite_data(wp_q_con, train_size=0.55, val_size=0.1, test_size=0.35)
# wp_seclected_df

In [35]:
df_list.append(wp_q_con)

In [36]:
concat_data = concat_df(df_list)
# concat_data

In [37]:
# clt_quantile_95
clt_q_con, _ = setA_setB(clt_quantile_95, concat_data)
# clt_seclected_df = splite_data(clt_q_con, train_size=0.55, val_size=0.1, test_size=0.35)
# clt_seclected_df

In [38]:
df_list.append(clt_q_con)

In [39]:
concat_data = concat_df(df_list)
concat_data

,Object_1,Relation,Object_2,Count,images,count_down
0,window,above,person,195,"[dresden_00473, leipzig_00188, amsterdam_00916...",0
1,wire,above,man,153,"[budapest_00322, potsdam_00075, brno_00672, br...",0
2,light,above,person,118,"[bologna_00606, prague_00231, toulouse_00517, ...",0
3,person,watching,light,94,"[leipzig_00416, barcelona_01095, potsdam_00337...",0
4,person,wearing,helmet,85,"[barcelona_00145, roma_00978, berlin_00096, fi...",0
...,...,...,...,...,...,...
85,man,watching,letter,16,"[set00_V014_458, set00_V007_465, set00_V009_61...",6
86,windshield,of,person,12,"[set02_V009_1256, set03_V009_17, set02_V009_56...",8
87,people,sitting on,table,10,"[set00_V012_1200, set00_V007_1275, set00_V014_...",15
88,people,under,light,10,"[set00_V013_1189, set00_V008_396, set00_V010_1...",15


In [40]:
ecp_seclected_tupel = splite_data(df_list[0], train_size=0.55, val_size=0.1, test_size=0.35)
len(ecp_seclected_tupel[0])

311

In [41]:
orig_human_im_list = list(set([item for sublist in r_cityperson_df['images'].tolist() for item in sublist]))
len(set(orig_human_im_list))

1926

In [22]:
def add_area(df):
    df = copy.deepcopy(df)
    df['area'] = ""
    for idx, row in df.iterrows():
        bb = df.loc[df['id'] == idx + 1, 'bbox']
        bb = bb.tolist()[0]
        df.loc[df['id'] == idx + 1, 'area'] = bb[2]*bb[3]
    return df

In [23]:
def splite_dataset(data_df, train_size=50, test_size=10, dt=None):
    if not dt:
        all_images = [el.rsplit('/', 1)[1] for el in data_df['file_name'].to_list()]
    else:
        all_images = data_df['file_name'].to_list()
    train_images = random.sample(all_images, train_size)
    rem_images = list(set(all_images).difference(set(train_images)))
    test_images = random.sample(rem_images, test_size)
    assert len(set(train_images).intersection(set(test_images))) == 0
    return train_images, test_images

# create annotatuion file
#### selected data frames are:
ecp_quantile_95

ch_q_ecp_q

wp_q_con

clt_q_con

concat_data

### select %15 from each validation and train set for testing

In [584]:
cp_file = 'json_files/CityPersons/train.json'
f = open(cp_file)
cp_data = json.load(f)
cp_licenses = cp_data['licenses']
cp_categories = cp_data['categories']
cp_images = cp_data['images']
cp_annotations = cp_data['annotations']
cp_img_df = pd.DataFrame(cp_images)
cp_ann_df = pd.DataFrame(cp_annotations)
cp_ann_df = add_area(cp_ann_df)
cp_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
0,1,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540
2,3,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72
3,4,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760
4,5,2,1,False,"[491.0, 396.0, 65.0, 158.0]",2048,1024,10270
...,...,...,...,...,...,...,...,...
27765,27766,2975,1,True,"[1558.0, 380.0, 18.0, 43.0]",2048,1024,774
27766,27767,2975,1,True,"[1679.0, 361.0, 27.0, 59.0]",2048,1024,1593
27767,27768,2975,1,True,"[416.0, 300.0, 15.0, 58.0]",2048,1024,870
27768,27769,2975,1,True,"[446.0, 351.0, 8.0, 65.0]",2048,1024,520


In [585]:
cp_img_df.shape[0]

2975

In [586]:
print(cp_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [587]:
for idx, row in cp_img_df.iterrows():
    cp_img_df.at[cp_img_df.index[idx], 'file_name'] = 'CityPersons/' + cp_img_df.at[cp_img_df.index[idx], 'file_name']


In [450]:
# test:  0.15 * 2975 = 446, train: 2975 - 446 = 2529
# test:  0.15 * 500 = 75,   val: 500 - 75 = 425

In [451]:
train_img_cp, test_img_cp_from_train = splite_dataset(cp_img_df, train_size=2529, test_size=446)
index_id_list = find_index_of_images(cp_img_df, image_name_list=train_img_cp)
train_cp_img_df = create_img_df(cp_img_df, index_id_list)
train_cp_ann_df = create_ann_df(cp_ann_df, index_id_list)
train_cp_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
7777,7778,991,1,False,"[285.0, 403.0, 28.0, 67.0]",2048,1024,1876
7778,7779,991,1,True,"[964.0, 419.0, 11.0, 27.0]",2048,1024,297
7779,7780,991,1,True,"[1092.0, 408.0, 15.0, 37.0]",2048,1024,555
7780,7781,991,1,True,"[260.0, 354.0, 13.0, 19.0]",2048,1024,247
7781,7782,991,1,True,"[327.0, 442.0, 18.0, 18.0]",2048,1024,324
...,...,...,...,...,...,...,...,...
3860,3861,483,1,True,"[1031.0, 381.0, 9.0, 20.0]",2048,1024,180
3861,3862,483,1,True,"[1044.0, 382.0, 7.0, 19.0]",2048,1024,133
3862,3863,483,1,True,"[766.0, 351.0, 6.0, 8.0]",2048,1024,48
3863,3864,483,1,True,"[1074.0, 348.0, 6.0, 9.0]",2048,1024,54


In [559]:
def create_test_dfs(in_img_df, in_ann_df, image_name_list):
    index_id_list = find_index_of_images(in_img_df, image_name_list=image_name_list)
    _img_df = create_img_df(in_img_df, index_id_list)
    _ann_df = create_ann_df(in_ann_df, index_id_list)
    _img_df = _img_df.sort_values('id')
    _img_df.reset_index(drop=True, inplace=True)
    
    for idx, row in _img_df.iterrows():
        img_id = _img_df.at[_img_df.index[idx], 'id']
        _img_df.at[_img_df.index[idx], 'id'] = idx + 1
        print(f"try replace {img_id} to {idx + 1} in anno")
        if 'image_id' in _ann_df.columns:
            _ann_df.loc[_ann_df['image_id'] == img_id, 'image_id'] = idx + 1
    
    _ann_df = _ann_df.sort_values('image_id')
    _ann_df.reset_index(drop=True, inplace=True)
    for idx, row in _ann_df.iterrows():
        _ann_df.at[_ann_df.index[idx], 'id'] = idx + 1
    return _img_df, _ann_df

In [583]:
im_df = copy.deepcopy(train_cp_img_df)
ann_df = copy.deepcopy(train_cp_ann_df)

In [453]:
train_cp_ann_df.loc[train_cp_ann_df['image_id'] == 2116]

,id,image_id,category_id,iscrowd,bbox,width,height,area


In [454]:
im_df = im_df.sort_values('id')
im_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
5,6,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
...,...,...,...,...,...,...,...,...
2970,2971,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2971,2972,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2972,2973,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2973,2974,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,


In [455]:
im_df.reset_index(drop=True, inplace=True)
im_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
3,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
4,6,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
...,...,...,...,...,...,...,...,...
2524,2971,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2525,2972,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2526,2973,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2527,2974,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,


In [456]:
for idx, row in im_df.iterrows():
    img_id = im_df.at[im_df.index[idx], 'id']
    im_df.at[im_df.index[idx], 'id'] = idx + 1
    print(f"try replace {img_id} to {idx + 1} in anno")
    if 'image_id' in ann_df.columns:
        ann_df.loc[ann_df['image_id'] == img_id, 'image_id'] = idx + 1

im_df

try replace 1 to 1 in anno
try replace 2 to 2 in anno
try replace 3 to 3 in anno
try replace 5 to 4 in anno
try replace 6 to 5 in anno
try replace 7 to 6 in anno
try replace 8 to 7 in anno
try replace 10 to 8 in anno
try replace 11 to 9 in anno
try replace 13 to 10 in anno
try replace 14 to 11 in anno
try replace 15 to 12 in anno
try replace 16 to 13 in anno
try replace 17 to 14 in anno
try replace 18 to 15 in anno
try replace 19 to 16 in anno
try replace 20 to 17 in anno
try replace 22 to 18 in anno
try replace 24 to 19 in anno
try replace 25 to 20 in anno
try replace 26 to 21 in anno
try replace 27 to 22 in anno
try replace 28 to 23 in anno
try replace 29 to 24 in anno
try replace 30 to 25 in anno
try replace 31 to 26 in anno
try replace 32 to 27 in anno
try replace 33 to 28 in anno
try replace 35 to 29 in anno
try replace 36 to 30 in anno
try replace 37 to 31 in anno
try replace 40 to 32 in anno
try replace 41 to 33 in anno
try replace 42 to 34 in anno
try replace 44 to 35 in anno
t

try replace 482 to 417 in anno
try replace 483 to 418 in anno
try replace 484 to 419 in anno
try replace 485 to 420 in anno
try replace 486 to 421 in anno
try replace 487 to 422 in anno
try replace 489 to 423 in anno
try replace 490 to 424 in anno
try replace 491 to 425 in anno
try replace 492 to 426 in anno
try replace 493 to 427 in anno
try replace 494 to 428 in anno
try replace 495 to 429 in anno
try replace 496 to 430 in anno
try replace 498 to 431 in anno
try replace 499 to 432 in anno
try replace 500 to 433 in anno
try replace 501 to 434 in anno
try replace 502 to 435 in anno
try replace 503 to 436 in anno
try replace 504 to 437 in anno
try replace 505 to 438 in anno
try replace 506 to 439 in anno
try replace 507 to 440 in anno
try replace 508 to 441 in anno
try replace 509 to 442 in anno
try replace 510 to 443 in anno
try replace 511 to 444 in anno
try replace 512 to 445 in anno
try replace 513 to 446 in anno
try replace 514 to 447 in anno
try replace 515 to 448 in anno
try repl

try replace 982 to 843 in anno
try replace 983 to 844 in anno
try replace 984 to 845 in anno
try replace 985 to 846 in anno
try replace 986 to 847 in anno
try replace 987 to 848 in anno
try replace 988 to 849 in anno
try replace 989 to 850 in anno
try replace 991 to 851 in anno
try replace 992 to 852 in anno
try replace 993 to 853 in anno
try replace 995 to 854 in anno
try replace 996 to 855 in anno
try replace 997 to 856 in anno
try replace 998 to 857 in anno
try replace 999 to 858 in anno
try replace 1002 to 859 in anno
try replace 1003 to 860 in anno
try replace 1005 to 861 in anno
try replace 1006 to 862 in anno
try replace 1008 to 863 in anno
try replace 1009 to 864 in anno
try replace 1011 to 865 in anno
try replace 1012 to 866 in anno
try replace 1013 to 867 in anno
try replace 1014 to 868 in anno
try replace 1015 to 869 in anno
try replace 1016 to 870 in anno
try replace 1017 to 871 in anno
try replace 1018 to 872 in anno
try replace 1019 to 873 in anno
try replace 1021 to 874 

try replace 1485 to 1269 in anno
try replace 1486 to 1270 in anno
try replace 1488 to 1271 in anno
try replace 1489 to 1272 in anno
try replace 1490 to 1273 in anno
try replace 1491 to 1274 in anno
try replace 1493 to 1275 in anno
try replace 1494 to 1276 in anno
try replace 1496 to 1277 in anno
try replace 1497 to 1278 in anno
try replace 1498 to 1279 in anno
try replace 1499 to 1280 in anno
try replace 1500 to 1281 in anno
try replace 1501 to 1282 in anno
try replace 1502 to 1283 in anno
try replace 1503 to 1284 in anno
try replace 1504 to 1285 in anno
try replace 1505 to 1286 in anno
try replace 1506 to 1287 in anno
try replace 1507 to 1288 in anno
try replace 1508 to 1289 in anno
try replace 1509 to 1290 in anno
try replace 1510 to 1291 in anno
try replace 1511 to 1292 in anno
try replace 1512 to 1293 in anno
try replace 1513 to 1294 in anno
try replace 1514 to 1295 in anno
try replace 1515 to 1296 in anno
try replace 1516 to 1297 in anno
try replace 1518 to 1298 in anno
try replac

try replace 1983 to 1686 in anno
try replace 1984 to 1687 in anno
try replace 1985 to 1688 in anno
try replace 1986 to 1689 in anno
try replace 1987 to 1690 in anno
try replace 1988 to 1691 in anno
try replace 1989 to 1692 in anno
try replace 1991 to 1693 in anno
try replace 1993 to 1694 in anno
try replace 1995 to 1695 in anno
try replace 1996 to 1696 in anno
try replace 1997 to 1697 in anno
try replace 1998 to 1698 in anno
try replace 1999 to 1699 in anno
try replace 2000 to 1700 in anno
try replace 2001 to 1701 in anno
try replace 2003 to 1702 in anno
try replace 2004 to 1703 in anno
try replace 2005 to 1704 in anno
try replace 2006 to 1705 in anno
try replace 2007 to 1706 in anno
try replace 2008 to 1707 in anno
try replace 2009 to 1708 in anno
try replace 2010 to 1709 in anno
try replace 2011 to 1710 in anno
try replace 2012 to 1711 in anno
try replace 2013 to 1712 in anno
try replace 2014 to 1713 in anno
try replace 2015 to 1714 in anno
try replace 2017 to 1715 in anno
try replac

try replace 2428 to 2065 in anno
try replace 2429 to 2066 in anno
try replace 2431 to 2067 in anno
try replace 2432 to 2068 in anno
try replace 2434 to 2069 in anno
try replace 2435 to 2070 in anno
try replace 2436 to 2071 in anno
try replace 2437 to 2072 in anno
try replace 2438 to 2073 in anno
try replace 2439 to 2074 in anno
try replace 2440 to 2075 in anno
try replace 2441 to 2076 in anno
try replace 2442 to 2077 in anno
try replace 2443 to 2078 in anno
try replace 2444 to 2079 in anno
try replace 2445 to 2080 in anno
try replace 2446 to 2081 in anno
try replace 2447 to 2082 in anno
try replace 2448 to 2083 in anno
try replace 2449 to 2084 in anno
try replace 2450 to 2085 in anno
try replace 2451 to 2086 in anno
try replace 2452 to 2087 in anno
try replace 2453 to 2088 in anno
try replace 2454 to 2089 in anno
try replace 2455 to 2090 in anno
try replace 2457 to 2091 in anno
try replace 2459 to 2092 in anno
try replace 2461 to 2093 in anno
try replace 2465 to 2094 in anno
try replac

try replace 2884 to 2444 in anno
try replace 2885 to 2445 in anno
try replace 2886 to 2446 in anno
try replace 2887 to 2447 in anno
try replace 2888 to 2448 in anno
try replace 2889 to 2449 in anno
try replace 2891 to 2450 in anno
try replace 2892 to 2451 in anno
try replace 2893 to 2452 in anno
try replace 2894 to 2453 in anno
try replace 2895 to 2454 in anno
try replace 2896 to 2455 in anno
try replace 2898 to 2456 in anno
try replace 2899 to 2457 in anno
try replace 2900 to 2458 in anno
try replace 2901 to 2459 in anno
try replace 2902 to 2460 in anno
try replace 2903 to 2461 in anno
try replace 2904 to 2462 in anno
try replace 2905 to 2463 in anno
try replace 2906 to 2464 in anno
try replace 2907 to 2465 in anno
try replace 2908 to 2466 in anno
try replace 2909 to 2467 in anno
try replace 2910 to 2468 in anno
try replace 2913 to 2469 in anno
try replace 2914 to 2470 in anno
try replace 2915 to 2471 in anno
try replace 2916 to 2472 in anno
try replace 2917 to 2473 in anno
try replac

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
...,...,...,...,...,...,...,...,...
2524,2525,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2525,2526,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2526,2527,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2527,2528,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,


In [457]:
ann_df = ann_df.sort_values('image_id')
ann_df.reset_index(drop=True, inplace=True)
ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
0,3,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540
2,1,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325
3,6,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224
4,4,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760
...,...,...,...,...,...,...,...,...
23651,27755,2529,1,False,"[1662.0, 366.0, 26.0, 63.0]",2048,1024,1638
23652,27754,2529,1,False,"[1642.0, 370.0, 25.0, 61.0]",2048,1024,1525
23653,27753,2529,1,False,"[1633.0, 369.0, 21.0, 51.0]",2048,1024,1071
23654,27769,2529,1,True,"[446.0, 351.0, 8.0, 65.0]",2048,1024,520


In [458]:
for idx, row in ann_df.iterrows():
    ann_df.at[ann_df.index[idx], 'id'] = idx + 1
ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
0,1,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540
2,3,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325
3,4,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224
4,5,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760
...,...,...,...,...,...,...,...,...
23651,23652,2529,1,False,"[1662.0, 366.0, 26.0, 63.0]",2048,1024,1638
23652,23653,2529,1,False,"[1642.0, 370.0, 25.0, 61.0]",2048,1024,1525
23653,23654,2529,1,False,"[1633.0, 369.0, 21.0, 51.0]",2048,1024,1071
23654,23655,2529,1,True,"[446.0, 351.0, 8.0, 65.0]",2048,1024,520


In [459]:
for i in range(2529):
    if ann_df.loc[ann_df['image_id'] == i].empty:
        print(f'{i} is empty')

0 is empty
8 is empty
20 is empty
33 is empty
37 is empty
59 is empty
93 is empty
95 is empty
117 is empty
170 is empty
188 is empty
222 is empty
226 is empty
235 is empty
255 is empty
258 is empty
264 is empty
356 is empty
357 is empty
362 is empty
370 is empty
374 is empty
397 is empty
398 is empty
404 is empty
411 is empty
416 is empty
423 is empty
430 is empty
466 is empty
475 is empty
485 is empty
490 is empty
513 is empty
533 is empty
538 is empty
539 is empty
555 is empty
556 is empty
571 is empty
575 is empty
579 is empty
591 is empty
609 is empty
662 is empty
673 is empty
708 is empty
753 is empty
755 is empty
756 is empty
760 is empty
762 is empty
763 is empty
767 is empty
793 is empty
808 is empty
814 is empty
827 is empty
834 is empty
845 is empty
849 is empty
863 is empty
870 is empty
940 is empty
948 is empty
967 is empty
996 is empty
1002 is empty
1010 is empty
1058 is empty
1078 is empty
1085 is empty
1118 is empty
1145 is empty
1154 is empty
1178 is empty
1181 is empty

In [460]:
cp_info = copy.deepcopy(cp_data['info'])
cp_cat = copy.deepcopy(cp_data['categories'])
cp_data['categories']

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]

In [461]:
aug_img_df = copy.deepcopy(im_df)
ann_df_c = copy.deepcopy(ann_df)

In [462]:
print(aug_img_df.shape[0])
print(ann_df_c.shape[0])

2529
23656


### base dataset

In [463]:
aug_data_train = dict()
aug_data_train['info'] = cp_info
aug_data_train['categories'] = cp_cat
aug_data_train['images'] = aug_img_df.to_dict('records')
aug_data_train['annotations'] = ann_df_c.to_dict('records')
with open('datasets/cityperson_train.json', 'w') as fp:
    json.dump(aug_data_train, fp)

### test anno

In [588]:
_img_df, _ann_df = create_test_dfs(cp_img_df, cp_ann_df, test_img_cp_from_train)
_img_df

try replace 4 to 1 in anno
try replace 9 to 2 in anno
try replace 12 to 3 in anno
try replace 21 to 4 in anno
try replace 23 to 5 in anno
try replace 34 to 6 in anno
try replace 38 to 7 in anno
try replace 39 to 8 in anno
try replace 43 to 9 in anno
try replace 45 to 10 in anno
try replace 54 to 11 in anno
try replace 93 to 12 in anno
try replace 94 to 13 in anno
try replace 96 to 14 in anno
try replace 114 to 15 in anno
try replace 122 to 16 in anno
try replace 138 to 17 in anno
try replace 139 to 18 in anno
try replace 144 to 19 in anno
try replace 151 to 20 in anno
try replace 165 to 21 in anno
try replace 167 to 22 in anno
try replace 168 to 23 in anno
try replace 175 to 24 in anno
try replace 178 to 25 in anno
try replace 187 to 26 in anno
try replace 189 to 27 in anno
try replace 192 to 28 in anno
try replace 200 to 29 in anno
try replace 205 to 30 in anno
try replace 220 to 31 in anno
try replace 225 to 32 in anno
try replace 228 to 33 in anno
try replace 233 to 34 in anno
try r

try replace 2602 to 390 in anno
try replace 2607 to 391 in anno
try replace 2608 to 392 in anno
try replace 2610 to 393 in anno
try replace 2628 to 394 in anno
try replace 2631 to 395 in anno
try replace 2636 to 396 in anno
try replace 2637 to 397 in anno
try replace 2639 to 398 in anno
try replace 2646 to 399 in anno
try replace 2656 to 400 in anno
try replace 2657 to 401 in anno
try replace 2664 to 402 in anno
try replace 2671 to 403 in anno
try replace 2680 to 404 in anno
try replace 2686 to 405 in anno
try replace 2689 to 406 in anno
try replace 2706 to 407 in anno
try replace 2713 to 408 in anno
try replace 2722 to 409 in anno
try replace 2723 to 410 in anno
try replace 2725 to 411 in anno
try replace 2728 to 412 in anno
try replace 2729 to 413 in anno
try replace 2731 to 414 in anno
try replace 2751 to 415 in anno
try replace 2761 to 416 in anno
try replace 2762 to 417 in anno
try replace 2768 to 418 in anno
try replace 2774 to 419 in anno
try replace 2777 to 420 in anno
try repl

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
...,...,...,...,...,...,...,...,...
441,442,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
442,443,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
443,444,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
444,445,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,


In [662]:
_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
...,...,...,...,...,...,...,...,...
441,442,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
442,443,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
443,444,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,
444,445,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1,,


In [663]:
_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
0,1,1,1,False,"[1183.0, 404.0, 31.0, 75.0]",2048,1024,2325
1,2,1,1,False,"[425.0, 345.0, 71.0, 172.0]",2048,1024,12212
2,3,1,1,False,"[484.0, 338.0, 76.0, 185.0]",2048,1024,14060
3,4,2,1,True,"[1746.0, 340.0, 134.0, 327.0]",2048,1024,43818
4,5,2,1,True,"[1011.0, 428.0, 23.0, 55.0]",2048,1024,1265
...,...,...,...,...,...,...,...,...
4109,4110,446,1,False,"[1651.0, 397.0, 35.0, 83.0]",2048,1024,2905
4110,4111,446,1,False,"[1620.0, 393.0, 35.0, 87.0]",2048,1024,3045
4111,4112,446,1,True,"[1803.0, 344.0, 9.0, 13.0]",2048,1024,117
4112,4113,446,1,False,"[1567.0, 385.0, 37.0, 91.0]",2048,1024,3367


In [667]:
base_data_test = dict()
base_data_test['info'] = cp_info
base_data_test['categories'] = cp_cat
base_data_test['images'] = _img_df.to_dict('records')
base_data_test['annotations'] = _ann_df.to_dict('records')
with open('datasets/cityperson_test.json', 'w') as fp:
    json.dump(base_data_test, fp)

In [601]:
t_aug_img_df = copy.deepcopy(_img_df)
t_ann_df_c = copy.deepcopy(_ann_df)

In [602]:
t_aug_img_df.shape[0]

446

In [603]:
t_ann_df_c.shape[0]

4114

## ECP 

In [464]:
ecp_file = 'json_files/EuroCity/day_train_all.json'
f = open(ecp_file)
ecp_data = json.load(f)
ecp_licenses = ecp_data['licenses']
ecp_categories = ecp_data['categories']
ecp_images = ecp_data['images']
ecp_annotations = ecp_data['annotations']
ecp_img_df = pd.DataFrame(ecp_images)
ecp_ann_df = pd.DataFrame(ecp_annotations)
ecp_img_df.shape[0]

23892

In [465]:
print(ecp_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [466]:
for idx, row in ecp_img_df.iterrows():
    ecp_img_df.at[ecp_img_df.index[idx], 'file_name'] = 'EuroCity/' + ecp_img_df.at[ecp_img_df.index[idx], 'file_name']
ecp_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
1,2,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
2,3,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
3,4,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
4,5,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
...,...,...,...,...,...,...,...,...
23887,23888,EuroCity/ECP/day/img/train/potsdam/potsdam_004...,1920,1024,2019-11-03 07:44:18.143034,1,,
23888,23889,EuroCity/ECP/day/img/train/potsdam/potsdam_000...,1920,1024,2019-11-03 07:44:18.143034,1,,
23889,23890,EuroCity/ECP/day/img/train/potsdam/potsdam_005...,1920,1024,2019-11-03 07:44:18.143034,1,,
23890,23891,EuroCity/ECP/day/img/train/potsdam/potsdam_003...,1920,1024,2019-11-03 07:44:18.143034,1,,


In [620]:
im_list = list(set([item + '.png' for sublist in ecp_quantile_95['images'].to_list() for item in sublist]))
len(im_list)

567

In [467]:
train_img_ecp, val_img_ecp, test_img_ecp = splite_data(ecp_quantile_95, train_size=0.7, val_size=0.15, test_size=0.15)
index_id_list = find_index_of_images(ecp_img_df, image_name_list=train_img_ecp)

selected_ecp_img_df = create_img_df(ecp_img_df, index_id_list)
selected_ecp_ann_df = create_ann_df(ecp_ann_df, index_id_list)
print(selected_ecp_ann_df.shape[0])
print(selected_ecp_img_df.shape[0])

2870
396


In [468]:
print(len(train_img_ecp))
len(val_img_ecp)

396


86

In [554]:
len(test_img_ecp)

85

In [469]:
print(aug_img_df.shape[0])
aug_img_df, selected_ecp_ann_df = uodate_id_in_image_ann(selected_ecp_img_df, aug_img_df, selected_ecp_ann_df)
print(aug_img_df.shape[0])

2529
2925


In [470]:
selected_ecp_ann_df = add_area(selected_ecp_ann_df)
print(selected_ecp_ann_df.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ecp_ann_df)
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']
ann_df_c

2870


,id,image_id,category_id,iscrowd,bbox,width,height,area
0,1,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540
2,3,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325
3,4,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224
4,5,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760
...,...,...,...,...,...,...,...,...
26521,26522,2882,1,True,"[1105.0, 541.0, 31.0, 51.0]",1920,1024,1581
26522,26523,2882,1,True,"[1107.0, 509.0, 29.0, 75.0]",1920,1024,2175
26523,26524,2882,1,True,"[1325.0, 539.0, 80.0, 59.0]",1920,1024,4720
26524,26525,2882,1,True,"[1344.0, 506.0, 43.0, 92.0]",1920,1024,3956


In [473]:
print(ann_df.shape[0])

23656


In [575]:
def _ugment_df(in_img_df, in_ann_df, test_img_list, _aug_img_df, _ann_df_c):   
    
    index_id_list = find_index_of_images(in_img_df, image_name_list=test_img_list)
    s_ecp_img_df = create_img_df(in_img_df, index_id_list)
    print(f"you will add {s_ecp_img_df.shape[0]} to images")
    
    s_ecp_ann_df = create_ann_df(in_ann_df, index_id_list)
    print(f"you will add {s_ecp_ann_df.shape[0]} to anno")

    _aug_img_df, s_ecp_ann_df = uodate_id_in_image_ann(s_ecp_img_df, _aug_img_df, s_ecp_ann_df)
    s_ecp_ann_df = add_area(s_ecp_ann_df)

    _ann_df_c = concat_ann_df(_ann_df_c, s_ecp_ann_df)
    for idx, row in t_ann_df_c.iterrows():
        assert idx + 1 == row['id']
    return _aug_img_df, _ann_df_c

#### test anno

In [604]:
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
t_aug_img_df, t_ann_df_c = _ugment_df(ecp_img_df, ecp_ann_df, test_img_ecp, t_aug_img_df, t_ann_df_c)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

446
4114
you will add 85 to images
you will add 612 to anno
531
4726


### crowdHuman

In [474]:
ch_file = 'json_files/CrowdHuman/train.json'

f = open(ch_file)
ch_data = json.load(f)
# ch_licenses = ch_data['licenses']
ch_categories = ch_data['categories']
ch_images = ch_data['images']
ch_annotations = ch_data['annotations']
ch_img_df = pd.DataFrame(ch_images)
ch_ann_df = pd.DataFrame(ch_annotations)

In [475]:
print(ch_data['categories'])

[{'supercategory': 'none', 'id': 1, 'name': 'person'}, {'supercategory': 'none', 'id': 2, 'name': 'mask'}]


In [476]:
bb_df = copy.deepcopy(ch_ann_df[ch_ann_df.category_id == 1])
ch_ann_df = bb_df.copy()
ch_ann_df

,area,iscrowd,image_id,bbox,hbox,vbox,category_id,id,ignore
0,86523,0,1,"[61, 123, 191, 453]","[123, 129, 63, 64]","[62, 126, 154, 446]",1,1,0
1,92378,0,1,"[165, 95, 187, 494]","[214, 97, 58, 74]","[175, 95, 140, 487]",1,2,0
2,96135,0,1,"[236, 104, 195, 493]","[318, 109, 58, 68]","[260, 106, 170, 487]",1,3,0
3,85852,0,1,"[452, 110, 169, 508]","[486, 119, 61, 74]","[455, 113, 141, 501]",1,4,0
4,62103,0,1,"[520, 95, 163, 381]","[559, 105, 53, 57]","[553, 98, 70, 118]",1,5,0
...,...,...,...,...,...,...,...,...,...
438781,286612,0,15000,"[3393, 590, 316, 907]","[3518, 595, 114, 133]","[3393, 590, 316, 907]",1,438782,0
438782,81890,0,15000,"[3448, 607, 190, 431]","[3511, 610, 93, 104]","[3452, 608, 184, 430]",1,438783,0
438783,240570,0,15000,"[3684, 554, 297, 810]","[3824, 559, 96, 105]","[3795, 559, 156, 794]",1,438784,0
438784,234855,0,15000,"[3797, 536, 255, 921]","[3909, 539, 89, 113]","[3889, 535, 112, 169]",1,438785,0


In [477]:
for idx, row in ch_img_df.iterrows():
    ch_img_df.at[ch_img_df.index[idx], 'file_name'] = 'CrowdHuman/Images/' + ch_img_df.at[ch_img_df.index[idx], 'file_name']
ch_img_df

,file_name,height,width,id
0,"CrowdHuman/Images/284193,faa9000f2678b5e.jpg",683,1024,1
1,"CrowdHuman/Images/273275,cd061000af95f691.jpg",480,600,2
2,"CrowdHuman/Images/273278,8d231000e09fc133.jpg",1017,1300,3
3,"CrowdHuman/Images/283554,2cd4d0007833968d.jpg",954,1300,4
4,"CrowdHuman/Images/283554,37ba1000a6ca8c4d.jpg",533,800,5
...,...,...,...,...
14995,"CrowdHuman/Images/273275,874d9000417e16ed.jpg",1192,1800,14996
14996,"CrowdHuman/Images/273278,d329e000260a8cc2.jpg",600,1800,14997
14997,"CrowdHuman/Images/282555,c5fd20007faf5f84.jpg",575,1024,14998
14998,"CrowdHuman/Images/273275,10b78d0006d7d7b9c.jpg",462,1838,14999


In [478]:
ch_selected_df = copy.deepcopy(ch_q_ecp_q)

In [479]:
ch_img_df

,file_name,height,width,id
0,"CrowdHuman/Images/284193,faa9000f2678b5e.jpg",683,1024,1
1,"CrowdHuman/Images/273275,cd061000af95f691.jpg",480,600,2
2,"CrowdHuman/Images/273278,8d231000e09fc133.jpg",1017,1300,3
3,"CrowdHuman/Images/283554,2cd4d0007833968d.jpg",954,1300,4
4,"CrowdHuman/Images/283554,37ba1000a6ca8c4d.jpg",533,800,5
...,...,...,...,...
14995,"CrowdHuman/Images/273275,874d9000417e16ed.jpg",1192,1800,14996
14996,"CrowdHuman/Images/273278,d329e000260a8cc2.jpg",600,1800,14997
14997,"CrowdHuman/Images/282555,c5fd20007faf5f84.jpg",575,1024,14998
14998,"CrowdHuman/Images/273275,10b78d0006d7d7b9c.jpg",462,1838,14999


In [480]:
train_img_ch, val_img_ch, test_img_ch = splite_data(ch_selected_df, train_size=0.7, val_size=0.15, test_size=0.15)
index_id_list = find_index_of_images(ch_img_df, image_name_list=train_img_ch)
selected_ch_img_df = create_img_df(ch_img_df, index_id_list)
selected_ch_ann_df = create_ann_df(ch_ann_df, index_id_list)
print(selected_ch_img_df.shape[0])
selected_ch_ann_df.shape[0]

566


8903

In [555]:
len(test_img_ch)

121

In [481]:
print(len(train_img_ch))
len(val_img_ch)

566


122

In [482]:
print(aug_img_df.shape[0])
aug_img_df, selected_ch_ann_df = uodate_id_in_image_ann(selected_ch_img_df, aug_img_df, selected_ch_ann_df, dt='ch')
aug_img_df.shape[0]

2925


3491

In [558]:
selected_ch_ann_df.shape[0]

8903

In [483]:
selected_ch_ann_df = add_area(selected_ch_ann_df)
print(ann_df_c.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ch_ann_df)
ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
ann_df_c

26526


,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,1,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,NaN
1,2,1,1,1,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,NaN
2,3,1,1,1,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,NaN
3,4,2,1,0,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,NaN
4,5,2,1,0,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,NaN
...,...,...,...,...,...,...,...,...,...
35424,35425,3063,1,0,"[3583, 1407, 521, 969]",4800,3200,504849,0.0
35425,35426,3063,1,0,"[3806, 1510, 727, 1421]",4800,3200,1033067,0.0
35426,35427,3063,1,0,"[4421, 1410, 396, 1077]",4800,3200,426492,0.0
35427,35428,3063,1,0,"[4522, 1522, 379, 970]",4800,3200,367630,0.0


#### test anno

In [605]:
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
t_aug_img_df, t_ann_df_c = _ugment_df(ch_img_df, ch_ann_df, test_img_ch, t_aug_img_df, t_ann_df_c)
t_ann_df_c = t_ann_df_c.drop(['hbox', 'vbox'], axis = 1)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

531
4726
you will add 121 to images
you will add 1571 to anno
652
6297


### wheelchair

In [484]:
wheel_file = 'json_files/wheelchair/train.json'

f = open(wheel_file)
wheel_data = json.load(f)
wheel_categories = wheel_data['categories']
wheel_images = wheel_data['images']
wheel_annotations = wheel_data['annotations']
wheel_img_df = pd.DataFrame(wheel_images)
wheel_ann_df = pd.DataFrame(wheel_annotations)
print(wheel_img_df.shape[0])

11


In [485]:
wheel_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,wheelchair/images/train/14b2ccfde47a3703.jpg,1024,683,2021-04-14 21:13:32.226876,1,,
1,2,wheelchair/images/train/b4df8bd4569e2bb5.jpg,683,1024,2021-04-14 21:13:32.226876,1,,
2,3,wheelchair/images/train/465e2e0e681f9ddc.jpg,678,1024,2021-04-14 21:13:32.226876,1,,
3,4,wheelchair/images/train/04607b0bacc2be8c.jpg,1024,654,2021-04-14 21:13:32.226876,1,,
4,5,wheelchair/images/train/64833b8b7f1404e8.jpg,1024,768,2021-04-14 21:13:32.226876,1,,
5,6,wheelchair/images/train/81fe77bf2f6d5cf6.jpg,1024,810,2021-04-14 21:13:32.226876,1,,
6,7,wheelchair/images/train/b8ebcca85d76eaa1.jpg,1024,683,2021-04-14 21:13:32.226876,1,,
7,8,wheelchair/images/train/108d5a62ba569a28.jpg,1024,683,2021-04-14 21:13:32.226876,1,,
8,9,wheelchair/images/train/cc0815f73df3586a.jpg,1024,768,2021-04-14 21:13:32.226876,1,,
9,10,wheelchair/images/train/712694559f078391.jpg,1024,1014,2021-04-14 21:13:32.226876,1,,


In [486]:
print(wheel_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [487]:
wheel_ann_df = add_area(wheel_ann_df)

print(wheel_ann_df.shape[0])

12


In [488]:
selected_img_df = copy.deepcopy(wheel_img_df)
selected_ann_df = copy.deepcopy(wheel_ann_df)

In [489]:
print(aug_img_df.shape[0])
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)
print(aug_img_df.shape[0])
# aug_img_df = copy.deepcopy(wheel_img_df)

3491
3502


In [490]:
print(ann_df_c.shape[0])
print(selected_ann_df.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
# ann_df_c = copy.deepcopy(wheel_ann_df)
ann_df_c
print(ann_df_c.shape[0])

35429
12
35441


In [491]:
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

#### test anno

In [606]:
wheel_file = 'json_files/wheelchair/test.json'

f = open(wheel_file)
wt_data = json.load(f)
wt_categories = wheel_data['categories']
wt_images = wt_data['images']
wt_annotations = wt_data['annotations']
wt_img_df = pd.DataFrame(wt_images)
wt_ann_df = pd.DataFrame(wt_annotations)
print(wt_img_df.shape[0])
print(wt_ann_df.shape[0])

5
6


In [607]:
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
s_img_df = copy.deepcopy(wt_img_df)
s_ann_df = copy.deepcopy(wt_ann_df)
t_aug_img_df, s_ann_df = uodate_id_in_image_ann(s_img_df, t_aug_img_df, s_ann_df)
t_ann_df_c = concat_ann_df(t_ann_df_c, s_ann_df)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

652
6297
657
6303


### WP

In [492]:
wp1_file = 'json_files/WiderPerson/train.json'

f = open(wp1_file)
wp_data = json.load(f)
wp_categories = wp_data['categories']
wp_images = wp_data['images']
wp_annotations = wp_data['annotations']
wp_img_df = pd.DataFrame(wp_images)
wp_ann_df = pd.DataFrame(wp_annotations)
wp_img_df.shape[0]

8000

In [493]:
wp_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,WiderPerson/Images/000040.jpg,550,413,2021-03-16 23:06:50.130304,1,,
1,2,WiderPerson/Images/000042.jpg,551,350,2021-03-16 23:06:50.130304,1,,
2,3,WiderPerson/Images/000051.jpg,450,800,2021-03-16 23:06:50.130304,1,,
3,4,WiderPerson/Images/000081.jpg,600,395,2021-03-16 23:06:50.130304,1,,
4,5,WiderPerson/Images/000089.jpg,600,400,2021-03-16 23:06:50.130304,1,,
...,...,...,...,...,...,...,...,...
7995,7996,WiderPerson/Images/020544.jpg,397,255,2021-03-16 23:06:50.130304,1,,
7996,7997,WiderPerson/Images/020545.jpg,700,476,2021-03-16 23:06:50.130304,1,,
7997,7998,WiderPerson/Images/020547.jpg,1272,784,2021-03-16 23:06:50.130304,1,,
7998,7999,WiderPerson/Images/020548.jpg,4208,2368,2021-03-16 23:06:50.130304,1,,


In [494]:
print(wp_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [495]:
train_img_wp, val_img_wp, test_img_wp = splite_data(wp_q_con, train_size=0.7, val_size=0.15, test_size=0.15)
index_id_list = find_index_of_images(wp_img_df, image_name_list=train_img_wp)

In [496]:
len(set(train_img_wp))

55

In [497]:
len(set(val_img_wp))

12

In [556]:
len(set(test_img_wp))

12

In [498]:
selected_wp_img_df = create_img_df(wp_img_df, index_id_list)
selected_wp_img_df.shape[0]

55

In [499]:
selected_wp_ann_df = create_ann_df(wp_ann_df, index_id_list)
selected_wp_ann_df = add_area(selected_wp_ann_df)
selected_wp_ann_df.shape[0]

1934

In [500]:
print(aug_img_df.shape[0])
aug_img_df, selected_wp_ann_df = uodate_id_in_image_ann(selected_wp_img_df, aug_img_df, selected_wp_ann_df)

print(aug_img_df.shape[0])

3502
3557


In [501]:
selected_wp_ann_df.shape[0]

1934

In [502]:
ann_df_c = concat_ann_df(ann_df_c, selected_wp_ann_df)
# ann_df_c

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

In [503]:
ann_df_c.shape[0]

37375

#### test anno

In [608]:
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
t_aug_img_df, t_ann_df_c = _ugment_df(wp_img_df, wp_ann_df, test_img_wp, t_aug_img_df, t_ann_df_c)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

657
6303
you will add 12 to images
you will add 441 to anno
669
6744


In [609]:
im_list = [item + '.png' for sublist in clt_q_con['images'].to_list() for item in sublist]
im_list

['set00_V014_158.png',
 'set00_V009_43.png',
 'set00_V012_1200.png',
 'set00_V007_1485.png',
 'set00_V007_285.png',
 'set00_V005_1768.png',
 'set00_V009_133.png',
 'set00_V014_488.png',
 'set00_V008_816.png',
 'set00_V007_255.png',
 'set00_V010_1641.png',
 'set00_V008_246.png',
 'set00_V013_1489.png',
 'set00_V014_248.png',
 'set00_V008_216.png',
 'set00_V007_1575.png',
 'set00_V008_6.png',
 'set00_V014_128.png',
 'set00_V014_1538.png',
 'set00_V013_1429.png',
 'set00_V014_458.png',
 'set00_V007_465.png',
 'set00_V009_613.png',
 'set00_V008_846.png',
 'set00_V007_495.png',
 'set00_V013_589.png',
 'set00_V009_1573.png',
 'set05_V000_1140.png',
 'set00_V010_171.png',
 'set00_V013_1219.png',
 'set00_V008_306.png',
 'set00_V009_673.png',
 'set00_V007_1785.png',
 'set00_V008_756.png',
 'set02_V009_1256.png',
 'set03_V009_17.png',
 'set02_V009_56.png',
 'set00_V014_1058.png',
 'set00_V006_865.png',
 'set02_V009_1196.png',
 'set00_V014_1328.png',
 'set00_V014_998.png',
 'set04_V003_475.png',


#### selected caltech

In [129]:
clt_q_con

,Object_1,Relation,Object_2,Count,images,count_down
2,window,behind,people,26,"[set00_V014_158, set00_V009_43, set00_V012_120...",0
4,man,watching,letter,16,"[set00_V014_458, set00_V007_465, set00_V009_61...",6
6,windshield,of,person,12,"[set02_V009_1256, set03_V009_17, set02_V009_56...",8
7,people,sitting on,table,10,"[set00_V012_1200, set00_V007_1275, set00_V014_...",15
8,people,under,light,10,"[set00_V013_1189, set00_V008_396, set00_V010_1...",15
10,person,sitting on,table,8,"[set00_V014_218, set00_V014_248, set00_V007_15...",16


In [141]:
ca_file = 'json_files/Caltech/train.json'

f = open(ca_file)
cl_data = json.load(f)
cl_categories = cl_data['categories']
cl_images = cl_data['images']
cl_annotations = cl_data['annotations']
cl_img_df = pd.DataFrame(cl_images)
cl_ann_df = pd.DataFrame(cl_annotations)
cl_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,set00_V000_I00002.jpg,640,480,2019-09-04 07:22:27.077839,1,,
1,2,set00_V000_I00005.jpg,640,480,2019-09-04 07:22:27.077839,1,,
2,3,set00_V000_I00008.jpg,640,480,2019-09-04 07:22:27.077839,1,,
3,4,set00_V000_I00011.jpg,640,480,2019-09-04 07:22:27.077839,1,,
4,5,set00_V000_I00014.jpg,640,480,2019-09-04 07:22:27.077839,1,,
...,...,...,...,...,...,...,...,...
42777,42778,set05_V012_I01694.jpg,640,480,2019-09-04 07:22:27.077839,1,,
42778,42779,set05_V012_I01697.jpg,640,480,2019-09-04 07:22:27.077839,1,,
42779,42780,set05_V012_I01700.jpg,640,480,2019-09-04 07:22:27.077839,1,,
42780,42781,set05_V012_I01703.jpg,640,480,2019-09-04 07:22:27.077839,1,,


In [143]:
from pathlib import Path
cal_im_path = 'datasets/Caltech/images/'
for el in cl_img_df['file_name'].to_list():
    im_name, _ = el.rsplit('.', 1)
    l, r = im_name.rsplit('_', 1)
    name_in_my_pc = l + '_' + str(int(r[1:])) + '.png'
    if not Path(f"{cal_im_path}{name_in_my_pc}").is_file():
        print(f"{name_in_my_pc} does not exist")
    

In [144]:
im_list = [item + '.png' for sublist in clt_q_con['images'].to_list() for item in sublist]
for im in im_list:
    if not Path(f"{cal_im_path}{im}").is_file():
        print(f"{im} does not exist")

In [145]:
for idx, row in cl_img_df.iterrows():
    file_name = cl_img_df.at[cl_img_df.index[idx], 'file_name']
    im_name, _ = file_name.rsplit('.', 1)
    l, r = im_name.rsplit('_', 1)
    cl_img_df.at[cl_img_df.index[idx], 'file_name'] =  l + '_' + str(int(r[1:])) + '.png'

for idx, row in cl_img_df.iterrows():
    cl_img_df.at[cl_img_df.index[idx], 'file_name'] = 'Caltech/images/' + cl_img_df.at[cl_img_df.index[idx], 'file_name']

cl_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,Caltech/images/set00_V000_2.png,640,480,2019-09-04 07:22:27.077839,1,,
1,2,Caltech/images/set00_V000_5.png,640,480,2019-09-04 07:22:27.077839,1,,
2,3,Caltech/images/set00_V000_8.png,640,480,2019-09-04 07:22:27.077839,1,,
3,4,Caltech/images/set00_V000_11.png,640,480,2019-09-04 07:22:27.077839,1,,
4,5,Caltech/images/set00_V000_14.png,640,480,2019-09-04 07:22:27.077839,1,,
...,...,...,...,...,...,...,...,...
42777,42778,Caltech/images/set05_V012_1694.png,640,480,2019-09-04 07:22:27.077839,1,,
42778,42779,Caltech/images/set05_V012_1697.png,640,480,2019-09-04 07:22:27.077839,1,,
42779,42780,Caltech/images/set05_V012_1700.png,640,480,2019-09-04 07:22:27.077839,1,,
42780,42781,Caltech/images/set05_V012_1703.png,640,480,2019-09-04 07:22:27.077839,1,,


In [119]:
cl_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height
0,1,23,1,True,"[213, 141, 5, 9]",640,480
1,2,24,1,True,"[216, 140, 5, 10]",640,480
2,3,25,1,True,"[220, 139, 5, 12]",640,480
3,4,26,1,True,"[221, 138, 5, 12]",640,480
4,5,27,1,True,"[223, 138, 5, 12]",640,480
...,...,...,...,...,...,...,...
64058,64059,42625,1,True,"[293, 203, 13, 19]",640,480
64059,64060,42626,1,True,"[294, 203, 13, 19]",640,480
64060,64061,42627,1,True,"[294, 203, 13, 19]",640,480
64061,64062,42628,1,True,"[295, 203, 13, 19]",640,480


In [147]:
train_img_cl, val_img_cl, test_img_cl = splite_data(clt_q_con, train_size=0.7, val_size=0.15, test_size=0.15)
train_img_cl

['set00_V006_865',
 'set00_V007_1275',
 'set00_V007_255',
 'set00_V008_396',
 'set00_V014_248',
 'set00_V010_1641',
 'set00_V009_613',
 'set00_V008_846',
 'set00_V008_216',
 'set00_V007_1575',
 'set00_V009_43',
 'set00_V014_1058',
 'set00_V012_1200',
 'set00_V014_488',
 'set00_V008_6',
 'set00_V009_673',
 'set02_V009_176',
 'set00_V010_1521',
 'set02_V009_56',
 'set00_V008_816',
 'set00_V009_1573',
 'set00_V005_1768',
 'set03_V009_17',
 'set00_V008_306',
 'set00_V013_1219',
 'set02_V009_1196',
 'set00_V013_1489',
 'set00_V009_133',
 'set00_V014_1238',
 'set00_V014_1538',
 'set00_V014_1328',
 'set00_V013_1189',
 'set00_V014_158',
 'set00_V008_246',
 'set04_V003_475']

In [ ]:
index_id_list = find_index_of_images(cl_img_df, image_name_list=train_img_cl)

selected_cl_img_df = create_img_df(cl_img_df, index_id_list)
print(selected_cl_img_df.shape[0])
selected_cl_ann_df = create_ann_df(cl_ann_df, index_id_list)
selected_cl_ann_df = add_area(selected_cl_ann_df)
selected_cl_ann_df.shape[0]

In [505]:
ca_file = 'json_files/selected_caltech/train.json'

f = open(ca_file)
cl_data = json.load(f)
cl_categories = cl_data['categories']
cl_images = cl_data['images']
cl_annotations = cl_data['annotations']
cl_img_df = pd.DataFrame(cl_images)
cl_ann_df = pd.DataFrame(cl_annotations)
cl_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,selected_caltech/images/train/set00_V013_1429.png,640,480,2021-04-03 12:21:50.728982,1,,
1,2,selected_caltech/images/train/set00_V009_43.png,640,480,2021-04-03 12:21:50.728982,1,,
2,3,selected_caltech/images/train/set03_V009_17.png,640,480,2021-04-03 12:21:50.728982,1,,
3,4,selected_caltech/images/train/set00_V013_829.png,640,480,2021-04-03 12:21:50.728982,1,,
4,5,selected_caltech/images/train/set00_V010_171.png,640,480,2021-04-03 12:21:50.728982,1,,
5,6,selected_caltech/images/train/set00_V010_1521.png,640,480,2021-04-03 12:21:50.728982,1,,
6,7,selected_caltech/images/train/set00_V013_1189.png,640,480,2021-04-03 12:21:50.728982,1,,
7,8,selected_caltech/images/train/set00_V007_1485.png,640,480,2021-04-03 12:21:50.728982,1,,
8,9,selected_caltech/images/train/set00_V013_1219.png,640,480,2021-04-03 12:21:50.728982,1,,
9,10,selected_caltech/images/train/set00_V009_673.png,640,480,2021-04-03 12:21:50.728982,1,,


In [506]:
cl_ann_df = add_area(cl_ann_df)

In [507]:
selected_img_df = copy.deepcopy(cl_img_df)
selected_ann_df = copy.deepcopy(cl_ann_df)
aug_img_df.shape[0] + selected_img_df.shape[0]

3581

In [508]:
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)

aug_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
...,...,...,...,...,...,...,...,...
3576,3577,selected_caltech/images/train/set00_V014_158.png,640,480,2021-04-03 12:21:50.728982,1.0,,
3577,3578,selected_caltech/images/train/set00_V007_1785.png,640,480,2021-04-03 12:21:50.728982,1.0,,
3578,3579,selected_caltech/images/train/set00_V009_1573.png,640,480,2021-04-03 12:21:50.728982,1.0,,
3579,3580,selected_caltech/images/train/set00_V008_306.png,640,480,2021-04-03 12:21:50.728982,1.0,,


In [509]:
aug_img_df.shape[0]

3581

In [510]:
selected_ann_df.shape[0] + ann_df_c.shape[0]

37441

In [511]:
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,1,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,NaN
1,2,1,1,1,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,NaN
2,3,1,1,1,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,NaN
3,4,2,1,0,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,NaN
4,5,2,1,0,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,NaN
...,...,...,...,...,...,...,...,...,...
37436,37437,3579,1,0,"[382, 136, 38, 78]",640,480,2964,NaN
37437,37438,3580,1,0,"[16, 161, 62, 80]",640,480,4960,NaN
37438,37439,3580,1,0,"[103, 167, 37, 56]",640,480,2072,NaN
37439,37440,3580,1,0,"[398, 159, 47, 32]",640,480,1504,NaN


In [512]:
ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
ann_df_c['ignore'] = ann_df_c['ignore'].fillna(0)
# ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,0.0
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,0.0
2,3,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,0.0
3,4,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,0.0
4,5,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,0.0
...,...,...,...,...,...,...,...,...,...
37436,37437,3579,1,False,"[382, 136, 38, 78]",640,480,2964,0.0
37437,37438,3580,1,False,"[16, 161, 62, 80]",640,480,4960,0.0
37438,37439,3580,1,False,"[103, 167, 37, 56]",640,480,2072,0.0
37439,37440,3580,1,False,"[398, 159, 47, 32]",640,480,1504,0.0


In [513]:
ann_df_c[ann_df_c.category_id != 1]

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore


In [514]:
wheel_data['categories']

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]

#### test anno

In [610]:
ct_file = 'json_files/selected_caltech/test.json'

f = open(ct_file)
ct_data = json.load(f)
ct_categories = ct_data['categories']
ct_images = ct_data['images']
ct_annotations = ct_data['annotations']

ct_img_df = pd.DataFrame(ct_images)
ct_ann_df = pd.DataFrame(ct_annotations)

print(ct_img_df.shape[0])
print(ct_ann_df.shape[0])

print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
s_img_df = copy.deepcopy(ct_img_df)
s_ann_df = copy.deepcopy(ct_ann_df)

t_aug_img_df, s_ann_df = uodate_id_in_image_ann(s_img_df, t_aug_img_df, s_ann_df)
t_ann_df_c = concat_ann_df(t_ann_df_c, s_ann_df)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

6
21
669
6744
675
6765


In [613]:
t_ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,0,"[1183.0, 404.0, 31.0, 75.0]",2048.0,1024.0,2325,NaN
1,2,1,1,0,"[425.0, 345.0, 71.0, 172.0]",2048.0,1024.0,12212,NaN
2,3,1,1,0,"[484.0, 338.0, 76.0, 185.0]",2048.0,1024.0,14060,NaN
3,4,2,1,1,"[1746.0, 340.0, 134.0, 327.0]",2048.0,1024.0,43818,NaN
4,5,2,1,1,"[1011.0, 428.0, 23.0, 55.0]",2048.0,1024.0,1265,NaN
...,...,...,...,...,...,...,...,...,...
6760,6761,673,1,0,"[184, 169, 20, 35]",640.0,480.0,NaN,NaN
6761,6762,673,1,0,"[175, 153, 21, 51]",640.0,480.0,NaN,NaN
6762,6763,674,1,0,"[480, 153, 36, 98]",640.0,480.0,NaN,NaN
6763,6764,674,1,0,"[45, 172, 13, 50]",640.0,480.0,NaN,NaN


In [612]:
t_aug_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8...,2048,1024,2019-07-25 11:20:43.195846,1.0,,
...,...,...,...,...,...,...,...,...
670,671,selected_caltech/images/test/set00_V012_1200.png,640,480,2021-04-15 13:32:40.736571,1.0,,
671,672,selected_caltech/images/test/set00_V014_218.png,640,480,2021-04-15 13:32:40.736571,1.0,,
672,673,selected_caltech/images/test/set00_V013_1489.png,640,480,2021-04-15 13:32:40.736571,1.0,,
673,674,selected_caltech/images/test/set00_V013_589.png,640,480,2021-04-15 13:32:40.736571,1.0,,


In [617]:
aug_data_test = dict()
aug_data_test['info'] = cp_data['info']
aug_data_test['licenses'] = wheel_data['licenses']
aug_data_test['categories'] = wheel_data['categories']

aug_data_test['images'] = t_aug_img_df.to_dict('records')
aug_data_test['annotations'] = t_ann_df_c.to_dict('records')

with open('datasets/aug_test.json', 'w') as fp:
    json.dump(aug_data_test, fp)

### Save to Json

In [515]:
aug_data_train = dict()
aug_data_train['info'] = cp_data['info']
aug_data_train['licenses'] = wheel_data['licenses']
aug_data_train['categories'] = wheel_data['categories']

aug_data_train['images'] = aug_img_df.to_dict('records')
aug_data_train['annotations'] = ann_df_c.to_dict('records')

In [516]:
with open('datasets/cp_train.json', 'w') as fp:
    json.dump(aug_data_train, fp)

## Validation

In [517]:
cp_file = 'json_files/CityPersons/val_gt_for_mmdetction.json'
f = open(cp_file)
cp_data = json.load(f)
# cp_licenses = cp_data['licenses']
cp_categories = cp_data['categories']
cp_images = cp_data['images']
cp_annotations = cp_data['annotations']
cp_img_df = pd.DataFrame(cp_images)
cp_ann_df = pd.DataFrame(cp_annotations)

In [518]:
for idx, row in cp_img_df.iterrows():
    cp_img_df.at[cp_img_df.index[idx], 'file_name'] = 'CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/' + cp_img_df.at[cp_img_df.index[idx], 'file_name']
cp_img_df


,id,file_name,height,width
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
2,3,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
3,4,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
...,...,...,...,...
495,496,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
496,497,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
497,498,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
498,499,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048


In [519]:
cp_ann_df['height'] = ""
cp_ann_df['width'] = ""
for idx, row in cp_img_df.iterrows():            
    cp_ann_df.loc[cp_ann_df['image_id'] == idx + 1, 'height'] = cp_img_df.at[idx, 'height']
    cp_ann_df.loc[cp_ann_df['image_id'] == idx + 1, 'width'] = cp_img_df.at[idx, 'width']

In [520]:
cp_ann_df = cp_ann_df.drop(['vis_bbox', 'vis_ratio'], axis = 1)
cp_ann_df = add_area(cp_ann_df)
cp_ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,0,"[947, 406, 17, 40]",1024,2048,680
1,2,1,1,0,0,"[1157, 375, 41, 99]",1024,2048,4059
2,3,1,1,0,0,"[1195, 381, 35, 84]",1024,2048,2940
3,4,1,1,0,0,"[1223, 379, 38, 91]",1024,2048,3458
4,5,1,1,0,1,"[1839, 382, 31, 71]",1024,2048,2201
...,...,...,...,...,...,...,...,...,...
5790,5791,498,1,0,1,"[1351, 402, 28, 68]",1024,2048,1904
5791,5792,499,1,0,1,"[1370, 391, 26, 18]",1024,2048,468
5792,5793,499,1,0,1,"[1468, 338, 11, 13]",1024,2048,143
5793,5794,499,1,0,1,"[925, 434, 27, 18]",1024,2048,486


In [521]:
val_img_cp, test_img_cp_from_val = splite_dataset(cp_img_df, train_size=425, test_size=75)
index_id_list = find_index_of_images(cp_img_df, image_name_list=val_img_cp)

val_cp_img_df = create_img_df(cp_img_df, index_id_list)
val_cp_ann_df = create_ann_df(cp_ann_df, index_id_list)

val_cp_img_df

,id,file_name,height,width
240,241,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
493,494,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
368,369,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
223,224,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
390,391,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
...,...,...,...,...
243,244,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
346,347,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
174,175,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
435,436,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048


In [522]:
val_cp_ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
3583,3584,241,1,0,0,"[810, 416, 10, 22]",1024,2048,220
3584,3585,241,1,0,0,"[821, 411, 11, 26]",1024,2048,286
3585,3586,241,1,0,0,"[1357, 316, 75, 182]",1024,2048,13650
3586,3587,241,1,0,0,"[1313, 333, 72, 177]",1024,2048,12744
3587,3588,241,1,0,1,"[928, 408, 11, 25]",1024,2048,275
...,...,...,...,...,...,...,...,...,...
2252,2253,162,1,0,1,"[825, 380, 13, 32]",1024,2048,416
2253,2254,162,1,0,1,"[1417, 353, 23, 29]",1024,2048,667
2254,2255,162,1,0,1,"[1456, 351, 14, 29]",1024,2048,406
2255,2256,162,1,0,1,"[768, 385, 16, 19]",1024,2048,304


In [523]:
im_df = copy.deepcopy(val_cp_img_df)
ann_df = copy.deepcopy(val_cp_ann_df)

In [524]:
ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
3583,3584,241,1,0,0,"[810, 416, 10, 22]",1024,2048,220
3584,3585,241,1,0,0,"[821, 411, 11, 26]",1024,2048,286
3585,3586,241,1,0,0,"[1357, 316, 75, 182]",1024,2048,13650
3586,3587,241,1,0,0,"[1313, 333, 72, 177]",1024,2048,12744
3587,3588,241,1,0,1,"[928, 408, 11, 25]",1024,2048,275
...,...,...,...,...,...,...,...,...,...
2252,2253,162,1,0,1,"[825, 380, 13, 32]",1024,2048,416
2253,2254,162,1,0,1,"[1417, 353, 23, 29]",1024,2048,667
2254,2255,162,1,0,1,"[1456, 351, 14, 29]",1024,2048,406
2255,2256,162,1,0,1,"[768, 385, 16, 19]",1024,2048,304


In [525]:
im_df = im_df.sort_values('id')
im_df

,id,file_name,height,width
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
5,6,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
6,7,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
...,...,...,...,...
493,494,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
494,495,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
495,496,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
496,497,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048


In [526]:
val_cp_ann_df.loc[val_cp_ann_df['image_id'] == 294]

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area


In [527]:
im_df.reset_index(drop=True, inplace=True)
for idx, row in im_df.iterrows():
    img_id = im_df.at[im_df.index[idx], 'id']
    im_df.at[im_df.index[idx], 'id'] = idx + 1
    print(f"try replace {img_id} to {idx + 1} in anno")
    ann_df.loc[ann_df.image_id == img_id, 'image_id'] = idx + 1

im_df

try replace 1 to 1 in anno
try replace 2 to 2 in anno
try replace 5 to 3 in anno
try replace 6 to 4 in anno
try replace 7 to 5 in anno
try replace 8 to 6 in anno
try replace 9 to 7 in anno
try replace 11 to 8 in anno
try replace 12 to 9 in anno
try replace 13 to 10 in anno
try replace 15 to 11 in anno
try replace 17 to 12 in anno
try replace 19 to 13 in anno
try replace 20 to 14 in anno
try replace 21 to 15 in anno
try replace 23 to 16 in anno
try replace 24 to 17 in anno
try replace 25 to 18 in anno
try replace 26 to 19 in anno
try replace 27 to 20 in anno
try replace 28 to 21 in anno
try replace 29 to 22 in anno
try replace 30 to 23 in anno
try replace 31 to 24 in anno
try replace 32 to 25 in anno
try replace 33 to 26 in anno
try replace 34 to 27 in anno
try replace 35 to 28 in anno
try replace 36 to 29 in anno
try replace 37 to 30 in anno
try replace 39 to 31 in anno
try replace 40 to 32 in anno
try replace 42 to 33 in anno
try replace 43 to 34 in anno
try replace 44 to 35 in anno
t

try replace 434 to 373 in anno
try replace 435 to 374 in anno
try replace 436 to 375 in anno
try replace 437 to 376 in anno
try replace 438 to 377 in anno
try replace 439 to 378 in anno
try replace 440 to 379 in anno
try replace 442 to 380 in anno
try replace 443 to 381 in anno
try replace 444 to 382 in anno
try replace 446 to 383 in anno
try replace 447 to 384 in anno
try replace 448 to 385 in anno
try replace 449 to 386 in anno
try replace 450 to 387 in anno
try replace 451 to 388 in anno
try replace 453 to 389 in anno
try replace 454 to 390 in anno
try replace 455 to 391 in anno
try replace 456 to 392 in anno
try replace 458 to 393 in anno
try replace 459 to 394 in anno
try replace 460 to 395 in anno
try replace 462 to 396 in anno
try replace 463 to 397 in anno
try replace 466 to 398 in anno
try replace 467 to 399 in anno
try replace 468 to 400 in anno
try replace 469 to 401 in anno
try replace 470 to 402 in anno
try replace 471 to 403 in anno
try replace 473 to 404 in anno
try repl

,id,file_name,height,width
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
2,3,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
3,4,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
...,...,...,...,...
420,421,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
421,422,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
422,423,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048
423,424,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048


In [528]:
ann_df = ann_df.sort_values('image_id')
ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
8,9,1,1,0,1,"[963, 378, 6, 8]",1024,2048,48
5,6,1,1,0,1,"[1617, 354, 24, 31]",1024,2048,744
6,7,1,1,0,1,"[1940, 455, 30, 54]",1024,2048,1620
7,8,1,1,0,1,"[1150, 347, 11, 14]",1024,2048,154
9,10,1,1,0,1,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
5782,5783,424,1,0,0,"[818, 410, 13, 32]",1024,2048,416
5787,5788,425,1,0,0,"[1029, 423, 16, 41]",1024,2048,656
5788,5789,425,1,0,0,"[1386, 418, 22, 53]",1024,2048,1166
5789,5790,425,1,0,1,"[1000, 424, 14, 34]",1024,2048,476


In [529]:
for i in range(425):
    if ann_df.loc[ann_df['image_id'] == i].empty:
        print(f'{i} is empty')

0 is empty
7 is empty
255 is empty
256 is empty
257 is empty
259 is empty
262 is empty
268 is empty
335 is empty
350 is empty
368 is empty


In [530]:
ann_df.reset_index(drop=True, inplace=True)
ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,9,1,1,0,1,"[963, 378, 6, 8]",1024,2048,48
1,6,1,1,0,1,"[1617, 354, 24, 31]",1024,2048,744
2,7,1,1,0,1,"[1940, 455, 30, 54]",1024,2048,1620
3,8,1,1,0,1,"[1150, 347, 11, 14]",1024,2048,154
4,10,1,1,0,1,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
4968,5783,424,1,0,0,"[818, 410, 13, 32]",1024,2048,416
4969,5788,425,1,0,0,"[1029, 423, 16, 41]",1024,2048,656
4970,5789,425,1,0,0,"[1386, 418, 22, 53]",1024,2048,1166
4971,5790,425,1,0,1,"[1000, 424, 14, 34]",1024,2048,476


In [531]:
for idx, row in ann_df.iterrows():
    ann_df.at[ann_df.index[idx], 'id'] = idx + 1

In [532]:
ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,0,1,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,0,1,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,0,1,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,0,1,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
4968,4969,424,1,0,0,"[818, 410, 13, 32]",1024,2048,416
4969,4970,425,1,0,0,"[1029, 423, 16, 41]",1024,2048,656
4970,4971,425,1,0,0,"[1386, 418, 22, 53]",1024,2048,1166
4971,4972,425,1,0,1,"[1000, 424, 14, 34]",1024,2048,476


In [533]:
aug_img_df = copy.deepcopy(im_df)
ann_df_c = copy.deepcopy(ann_df)

In [534]:
## Base validation
aug_data_val = dict()
aug_data_val['info'] = cp_info
aug_data_val['categories'] = cp_cat
aug_data_val['images'] = aug_img_df.to_dict('records')
aug_data_val['annotations'] = ann_df_c.to_dict('records')
with open('datasets/cityperson_val.json', 'w') as fp:
    json.dump(aug_data_val, fp)

### ECP

In [535]:
index_id_list = find_index_of_images(ecp_img_df, image_name_list=val_img_ecp)

selected_img_df = create_img_df(ecp_img_df, index_id_list)

selected_ann_df = create_ann_df(ecp_ann_df, index_id_list)
selected_ann_df = add_area(selected_ann_df)
selected_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
13275,13276,1947,1,False,"[1260.0, 525.0, 25.0, 79.0]",1920,1024,1975
13276,13277,1947,1,False,"[1469.0, 536.0, 15.0, 37.0]",1920,1024,555
13277,13278,1947,1,True,"[1694.0, 535.0, 22.0, 20.0]",1920,1024,440
133239,133240,16893,1,False,"[1058.0, 545.0, 10.0, 34.0]",1920,1024,340
133240,133241,16893,1,False,"[1075.0, 542.0, 11.0, 36.0]",1920,1024,396
...,...,...,...,...,...,...,...,...
19916,19917,3145,1,False,"[926.0, 562.0, 27.0, 76.0]",1920,1024,2052
19917,19918,3145,1,False,"[1052.0, 579.0, 23.0, 61.0]",1920,1024,1403
19918,19919,3145,1,False,"[1072.0, 562.0, 30.0, 77.0]",1920,1024,2310
19919,19920,3145,1,False,"[1470.0, 537.0, 23.0, 69.0]",1920,1024,1587


In [536]:
print(aug_img_df.shape[0])
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)
aug_img_df.shape[0]

425


511

In [537]:
print(ann_df_c.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
print(ann_df_c.shape[0])
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

ann_df_c

4973
5712


,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1.0,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,0,1.0,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,0,1.0,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,0,1.0,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
5707,5708,438,1,0,NaN,"[926.0, 562.0, 27.0, 76.0]",1024,1920,2052
5708,5709,438,1,0,NaN,"[1052.0, 579.0, 23.0, 61.0]",1024,1920,1403
5709,5710,438,1,0,NaN,"[1072.0, 562.0, 30.0, 77.0]",1024,1920,2310
5710,5711,438,1,0,NaN,"[1470.0, 537.0, 23.0, 69.0]",1024,1920,1587


### CH

In [538]:
index_id_list = find_index_of_images(ch_img_df, image_name_list=val_img_ch)

selected_img_df = create_img_df(ch_img_df, index_id_list)
selected_ann_df = create_ann_df(ch_ann_df, index_id_list)
print(selected_img_df.shape[0])
print(selected_ann_df.shape[0])

selected_ann_df = add_area(selected_ann_df)

aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df, dt='ch')

print(ann_df_c.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
print(ann_df_c.shape[0])

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']
ann_df_c

122
2259
5712
7971


,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area,hbox,vbox
0,1,1,1,0,1.0,"[963, 378, 6, 8]",1024,2048,48,NaN,NaN
1,2,1,1,0,1.0,"[1617, 354, 24, 31]",1024,2048,744,NaN,NaN
2,3,1,1,0,1.0,"[1940, 455, 30, 54]",1024,2048,1620,NaN,NaN
3,4,1,1,0,1.0,"[1150, 347, 11, 14]",1024,2048,154,NaN,NaN
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7966,7967,594,1,0,0.0,"[334, 113, 111, 391]",600,769,43401,"[361, 118, 48, 65]","[334, 113, 111, 391]"
7967,7968,594,1,0,0.0,"[255, 220, 100, 275]",600,769,27500,"[281, 225, 41, 51]","[255, 220, 100, 275]"
7968,7969,594,1,0,0.0,"[229, 88, 133, 391]",600,769,52003,"[266, 91, 46, 63]","[229, 88, 133, 391]"
7969,7970,594,1,0,0.0,"[90, 92, 153, 442]",600,769,67626,"[132, 95, 56, 67]","[90, 92, 153, 442]"


In [539]:
ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
ann_df_c

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1.0,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,0,1.0,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,0,1.0,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,0,1.0,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
7966,7967,594,1,0,0.0,"[334, 113, 111, 391]",600,769,43401
7967,7968,594,1,0,0.0,"[255, 220, 100, 275]",600,769,27500
7968,7969,594,1,0,0.0,"[229, 88, 133, 391]",600,769,52003
7969,7970,594,1,0,0.0,"[90, 92, 153, 442]",600,769,67626


### wheelchair

In [540]:
wheel_file = 'json_files/wheelchair/val.json'

f = open(wheel_file)
wheel_data = json.load(f)
wheel_categories = wheel_data['categories']
wheel_images = wheel_data['images']
wheel_annotations = wheel_data['annotations']
wheel_img_df = pd.DataFrame(wheel_images)
wheel_ann_df = pd.DataFrame(wheel_annotations)
wheel_img_df.shape[0]

4

In [541]:
wheel_ann_df = add_area(wheel_ann_df)
selected_img_df = copy.deepcopy(wheel_img_df)
selected_ann_df = copy.deepcopy(wheel_ann_df)
print(aug_img_df.shape[0])
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)

aug_img_df.shape[0]

633


637

In [542]:
print(ann_df_c.shape[0])

ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

ann_df_c.shape[0]

7971


7975

## WP

In [543]:
index_id_list = find_index_of_images(wp_img_df, image_name_list=val_img_wp)

In [544]:
selected_wp_img_df = create_img_df(wp_img_df, index_id_list)
selected_wp_ann_df = create_ann_df(wp_ann_df, index_id_list)
selected_wp_ann_df = add_area(selected_wp_ann_df)
print(selected_wp_img_df.shape[0])
selected_wp_ann_df.shape[0]

12


494

In [545]:
print(aug_img_df.shape[0])
aug_img_df, selected_wp_ann_df = uodate_id_in_image_ann(selected_wp_img_df, aug_img_df, selected_wp_ann_df)
print(aug_img_df.shape[0])

637
649


In [546]:
print(ann_df_c.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_wp_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']
print(ann_df_c.shape[0])
ann_df_c

7975
8469


,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1.0,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,0,1.0,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,0,1.0,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,0,1.0,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
8464,8465,642,1,0,NaN,"[479.0, 83.0, 59.0, 147.0]",365,550,8673
8465,8466,642,1,0,NaN,"[485.0, 38.0, 54.0, 131.0]",365,550,7074
8466,8467,642,1,0,NaN,"[439.0, 22.0, 52.0, 126.0]",365,550,6552
8467,8468,642,1,0,NaN,"[513.0, 9.0, 37.0, 112.0]",365,550,4144


### selected_caltech

In [547]:
ca_file = 'json_files/selected_caltech/val.json'

f = open(ca_file)
cl_data = json.load(f)
cl_categories = cl_data['categories']
cl_images = cl_data['images']
cl_annotations = cl_data['annotations']
cl_img_df = pd.DataFrame(cl_images)
cl_ann_df = pd.DataFrame(cl_annotations)
print(cl_img_df.shape[0])
cl_ann_df = add_area(cl_ann_df)

print(cl_ann_df.shape[0])

6
18


In [548]:
print(aug_img_df.shape[0])
selected_img_df = copy.deepcopy(cl_img_df)
selected_ann_df = copy.deepcopy(cl_ann_df)

aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)

aug_img_df.shape[0]

649


655

In [549]:
print(selected_ann_df.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
ann_df_c['ignore'] = ann_df_c['ignore'].fillna(0)
ann_df_c

18


,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,False,1.0,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,False,1.0,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,False,1.0,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,False,1.0,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,False,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
8482,8483,654,1,False,0.0,"[577, 144, 21, 57]",480,640,1197
8483,8484,654,1,False,0.0,"[406, 157, 18, 38]",480,640,684
8484,8485,654,1,False,0.0,"[482, 154, 16, 41]",480,640,656
8485,8486,655,1,False,0.0,"[287, 159, 26, 45]",480,640,1170


In [550]:
aug_img_df['date_captured'] = ""
aug_img_df['license'] = ""
aug_img_df['coco_url'] = ""
aug_img_df['flickr_url'] = ""
aug_img_df

,id,file_name,height,width,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048,,,,
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048,,,,
2,3,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048,,,,
3,4,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048,,,,
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8b...,1024,2048,,,,
...,...,...,...,...,...,...,...,...
650,651,selected_caltech/images/val/set00_V007_255.png,480,640,,,,
651,652,selected_caltech/images/val/set00_V014_128.png,480,640,,,,
652,653,selected_caltech/images/val/set00_V014_998.png,480,640,,,,
653,654,selected_caltech/images/val/set00_V007_1575.png,480,640,,,,


In [551]:
ann_df_c[ann_df_c.category_id != 1]

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area


In [614]:
ann_df_c

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,False,1.0,"[963, 378, 6, 8]",1024,2048,48
1,2,1,1,False,1.0,"[1617, 354, 24, 31]",1024,2048,744
2,3,1,1,False,1.0,"[1940, 455, 30, 54]",1024,2048,1620
3,4,1,1,False,1.0,"[1150, 347, 11, 14]",1024,2048,154
4,5,1,1,False,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
8482,8483,654,1,False,0.0,"[577, 144, 21, 57]",480,640,1197
8483,8484,654,1,False,0.0,"[406, 157, 18, 38]",480,640,684
8484,8485,654,1,False,0.0,"[482, 154, 16, 41]",480,640,656
8485,8486,655,1,False,0.0,"[287, 159, 26, 45]",480,640,1170


In [552]:
cl_data['categories']

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]

### Save json file

In [553]:
aug_data_train = dict()
aug_data_train['info'] = cp_info
aug_data_train['categories'] = cl_data['categories']
aug_data_train['images'] = aug_img_df.to_dict('records')
aug_data_train['annotations'] = ann_df_c.to_dict('records')
with open('datasets/cp_val.json', 'w') as fp:
    json.dump(aug_data_train, fp)